In [46]:
import trimesh
import numpy as np
import sys
from io_utils import stdout_redirected
import time
from dvidutils import encode_faces_to_custom_drc_bytes
import dask

def get_face_indices_in_range(mesh, face_mins, stop):
    max_edge_length = np.max(mesh.edges_unique_length)
    rows = np.where(face_mins[:,0]<stop+max_edge_length)
    return rows[0]

def renumber_vertex_indices(faces, vertex_indices_in_range):
    def renumber_indicies(a, val_old, val_new):
        arr = np.empty(a.max()+1, dtype=val_new.dtype)
        arr[val_old] = val_new
        return arr[a]
    
    faces = np.reshape(faces,-1)
    faces=renumber_indicies(faces,vertex_indices_in_range,np.arange(len(vertex_indices_in_range)))

    return np.reshape(faces,(-1,3))

@dask.delayed
def generate_mesh_decomposition(v,f,x_start, x_end):
    # Scale our coordinates.
    # scale = nodes_per_dim/(maximum_coordinates-minimum_coordinates)
    # verts_scaled = scale*(verts - minimum_coordinates)

    # # Define plane normals and create a trimesh object.
    # mesh = trimesh.Trimesh(vertices=verts_scaled, faces=faces)

    # submeshes = []
    # nodes = []
    # ratio = nodes_per_dim/max_nodes_per_dim

    nyz, nxz, nxy = np.eye(3)
    ratio = 64*4
    max_nodes_per_dim = 234
    t = time.time()
    total_blocks = 0
    sz = 0
    for x in range(x_start, x_end+1):
        vx, fx = trimesh.intersections.slice_faces_plane(
            v, f, plane_normal=-nyz, plane_origin=nyz*(x+1)*ratio)
        for y in range(0, max_nodes_per_dim):
            vy, fy = trimesh.intersections.slice_faces_plane(
                vx, fx, plane_normal=-nxz, plane_origin=nxz*(y+1)*ratio)
            # vy, fy = trimesh.intersections.slice_faces_plane(
            #     vy, fy, plane_normal=nxz, plane_origin=nxz*y*ratio)
            for z in range(0, max_nodes_per_dim):
                vz, fz = trimesh.intersections.slice_faces_plane(
                    vy, fy, plane_normal=-nxy, plane_origin=nxy*(z+1)*ratio)
                if len(vz) > 0:
                    normals = np.zeros(np.shape(vz))
                    draco_bytes = normals
                    #draco_bytes = encode_faces_to_custom_drc_bytes(
                    #    vz, normals, fz, np.asarray(3*[ratio]), np.asarray([0, 0, 0]))

                if (len(vz) > 0):
                    total_blocks += 1
                    sz = max(sz, len(draco_bytes))

                vy, fy = trimesh.intersections.slice_faces_plane(
                    vy, fy, plane_normal=nxy, plane_origin=nxy*(z+1)*ratio)

            vx, fx = trimesh.intersections.slice_faces_plane(
                vx, fx, plane_normal=nxz, plane_origin=nxz*(y+1)*ratio)
        
        v, f = trimesh.intersections.slice_faces_plane(
            v, f, plane_normal=nyz, plane_origin=nyz*(x+1)*ratio)
    print((x, y, z, total_blocks, time.time()-t, sz))
    return (x, y, z, total_blocks, time.time()-t, sz)

In [6]:
mesh = trimesh.load(
        "/groups/cosem/cosem/ackermand/meshesForWebsite/res1decimation0p1/jrc_hela-1/er_seg/17.obj")
print("read")
print(np.min(mesh.vertices, axis=0))
print(np.max(mesh.vertices, axis=0))

read
[3.49597e+03 3.73832e+00 2.67659e+04]
[27938.8   5679.44 59501.7 ]


In [8]:
from dask.distributed import Client, progress
client = Client(threads_per_worker=4,
                n_workers=16)


distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/multiresolutionMeshes/dask-worker-space/worker-5pxy494t', purging
distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/multiresolutionMeshes/dask-worker-space/worker-4dp_nkig', purging
distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/multiresolutionMeshes/dask-worker-space/worker-na8qzu9y', purging
distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/multiresolutionMeshes/dask-worker-space/worker-ehpi4plq', purging
distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/multiresolutionMeshes/dask-worker-space/worker-5aex28y3', purging
distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ac

7.0


In [52]:
import dask
results = []
v = mesh.vertices
f = mesh.faces
#face_maxs = np.max(mesh.vertices[mesh.faces],axis=1)

nyz, nxz, nxy = np.eye(3)
box_size = 64*4
num_workers = 16
x_start = int(np.floor(np.min(mesh.vertices[:,0])/box_size))
x_end = int(np.ceil(np.max(mesh.vertices[:,0])/box_size))

x_stride = int(np.ceil(1.0*(x_end-x_start)/num_workers))
print(x_stride)
for x in range(x_start, x_end+1,x_stride):
    # before slicing, try to limit number of vertices
    #face_indices_in_range = get_face_indices_in_range(mesh, face_maxs, (x+1)*ratio)
    #vertex_indices_in_range = np.unique(mesh.faces[face_indices_in_range])
    #v_new = v[vertex_indices_in_range]
    #f_new = mesh.faces[face_indices_in_range]
    #f_new = renumber_vertex_indices(f_new, vertex_indices_in_range)

    vx, fx = trimesh.intersections.slice_faces_plane(
        v, f, plane_normal=-nyz, plane_origin=nyz*(x+x_stride)*box_size)
    print("sliced 1")
    big_future_vx = client.scatter(vx)
    big_future_fx = client.scatter(fx)
    results.append( generate_mesh_decomposition(vx, fx, x, x+x_stride) )

    v, f = trimesh.intersections.slice_faces_plane(
        v, f, plane_normal=nyz, plane_origin=nyz*(x+x_stride)*box_size)
    print(x,len(v), len(vx))
    #v, f = trimesh.intersections.slice_faces_plane(
    #    v, f, plane_normal=nyz, plane_origin=nyz*(x+1)*ratio)



7
sliced 1
13 7748465 9394
sliced 1
20 7711478 50580
sliced 1
27 7327510 447114
sliced 1
34 6743500 644101
sliced 1
41 6178618 642050
sliced 1
48 5527039 738118
sliced 1
55 4678963 962142
sliced 1
62 3575471 1226244
sliced 1
69 2460802 1237086
sliced 1
76 1361363 1203932
sliced 1
83 489481 928836
sliced 1
90 122156 382083
sliced 1
97 6451 118503
sliced 1
104 0 6451


In [53]:
temp = dask.compute(* results)
print(temp[0])

(111, 233, 233, 94, 4.401657342910767, 895)
(20, 233, 233, 112, 5.4398512840271, 1013)
(27, 233, 233, 694, 11.316652297973633, 969)
(104, 233, 233, 1051, 21.85186767578125, 1255)
(97, 233, 233, 2889, 57.94137263298035, 1456)
(34, 233, 233, 4234, 67.60677480697632, 1536)
(48, 233, 233, 4700, 88.35539984703064, 1439)
(41, 233, 233, 4937, 88.66699457168579, 1540)
(55, 233, 233, 5400, 99.10814189910889, 1640)
(62, 233, 233, 6626, 121.77773070335388, 1646)
(90, 233, 233, 6504, 122.05378484725952, 1721)
(83, 233, 233, 8046, 148.49018597602844, 1649)
(69, 233, 233, 7746, 152.5274794101715, 1600)
(76, 233, 233, 7858, 152.82385277748108, 1520)
(20, 233, 233, 112, 5.439925909042358, 1013)


In [54]:
total_time = 0
for t in temp:
    total_time += t[4]

print(total_time,total_time/16)

1142.362603187561 71.39766269922256


In [6]:
#np.max(mesh.edges_unique_length)
face_mins = np.min(mesh.vertices[mesh.faces],axis=1)
face_maxs = np.max(mesh.vertices[mesh.faces],axis=1)

In [7]:
rows = np.where((face_mins[:,0]>0) & 
(face_maxs[:,0]<50000) & 
(face_mins[:,1]>0) & 
(face_maxs[:,1]<50000) & 
(face_mins[:,2]>0) & 
(face_maxs[:,2]<50000))

In [32]:
np.min(face_mins[:,0])

TrackedArray(3495.97)

In [19]:
mesh.vertices[np.unique(mesh.faces[rows])]

TrackedArray([[ 7864.32,  2635.89, 41387.8 ],
              [ 7864.32,  1232.67, 43349.9 ],
              [13107.2 ,  4374.88, 34502.2 ],
              ...,
              [ 9212.87,  1372.16, 39911.4 ],
              [ 9796.55,  3542.41, 36529.2 ],
              [ 9796.55,  2712.98, 49511.6 ]])

In [55]:
face_indices_in_range = get_face_indices_in_range(mesh, face_mins, (24+1)*64*4)
print(len(face_indices_in_range))
print(np.max(mesh.edges_unique_length))
print(len(mesh.faces))
vertex_indices_in_range = np.unique(mesh.faces[face_indices_in_range])
v_new = mesh.vertices[vertex_indices_in_range]

f_new = mesh.faces[face_indices_in_range]
print("yo")


104555
490.09894196580495
15827909
yo


In [61]:
def renumber_indicies(a, val_old, val_new):
    arr = np.empty(a.max()+1, dtype=val_new.dtype)
    arr[val_old] = val_new
    return arr[a]

f_temp = np.reshape(f_new,-1)
f_temp=renumber_indicies(f_temp,vertex_indices_in_range,np.arange(len(vertex_indices_in_range)))

f_temp = np.reshape(f_temp,(-1,3))

[[7677359 1264586 1739669]
 [7732071 7677359 1739669]
 [7677359  539375 1264586]
 ...
 [7712171 3872239 3745982]
 [4794047 3745982 3872239]
 [6974700 3745982 4794047]]
7757066
52455
[[47793  4623  6339]
 [50966 47793  6339]
 [47793  2018  4623]
 ...
 [49742 26176 18457]
 [29783 18457 26176]
 [37520 18457 29783]]


In [10]:
# https://stackoverflow.com/questions/53351060/dask-shared-memory-in-parallel-model

from multiprocessing import Pool

with Pool(processes = 20) as Pool:
   list_of_result = Pool.map(generate_mesh_decomposition, list(range(23,234)))

(23, 233, 233, 136, 5.585337162017822, 2112)
(26, 233, 233, 283, 9.896867275238037, 2397)
(24, 233, 233, 166, 5.333194255828857, 2112)
(25, 233, 233, 204, 6.561883926391602, 2137)
(27, 233, 233, 374, 16.86083221435547, 2958)
(29, 233, 233, 667, 36.13961863517761, 4213)
(28, 233, 233, 520, 33.232218980789185, 3423)


Process ForkPoolWorker-60:
Process ForkPoolWorker-50:
Process ForkPoolWorker-48:
Process ForkPoolWorker-59:
Process ForkPoolWorker-57:
Process ForkPoolWorker-65:


KeyboardInterrupt: 

In [9]:
import multiprocessing
print( multiprocessing.cpu_count())


20
